In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.decomposition import PCA

# Load the dataset
energy_data = pd.read_csv('energy_weather101.csv')

# Check the shape of the dataset
print(energy_data.shape)

"""## Data Exploration"""

print("Displaying the first 5 rows","\n")
print(energy_data.head())

print("Displaying the last 5 rows","\n")
print(energy_data.tail())

print("Get info on the dataset")
energy_data.info()

print("Descriptive statistics","\n")
print(energy_data.describe(), "\n")

# Check for null values
print("Columns with null values","\n")
print(energy_data.isnull().sum())

# Remove unnecessary columns
energy_data.drop(['generation hydro pumped storage aggregated','forecast wind offshore eday ahead'], axis=1, inplace=True)

# Replace null values with column means
energy_data.fillna(energy_data.mean(), inplace=True)

# Feature Selection

# Delete columns starting with 'generation'
energy_data.drop(energy_data.filter(regex='generation'), axis=1, inplace=True)

# Remove specific columns
energy_data.drop(['rain_1h','rain_3h','snow_3h','clouds_all'], axis=1, inplace=True)

# Check for nan values
print("Columns with null values after processing","\n")
print(energy_data.isnull().sum())

# Replace unique items in 'weather_main' with numbers
unique_weather_main = energy_data['weather_main'].unique()
weather_main_ranks = pd.Series(range(1, len(unique_weather_main)+1), index=unique_weather_main)
energy_data['weather_main'] = energy_data['weather_main'].map(weather_main_ranks)

# Remove 'weather_description' and 'weather_icon' columns
energy_data.drop(['weather_description', 'weather_icon'], axis=1, inplace=True)

# Remove columns irrelevant to analysis
energy_data.drop(['forecast solar day ahead', 'forecast wind onshore day ahead', 'total load forecast', 'price day ahead', 'price actual', 'city_name'], axis=1, inplace=True)

# Data Visualization

# Calculate autocorrelation
lags = np.arange(1, 24 * 10 + 1)
autocorrelation = [energy_data['total load actual'].autocorr(lag=l) for l in lags]

# Plot autocorrelation
plt.figure()
plt.plot(lags, autocorrelation)
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.title("Autocorrelation of Actual Total Load")
plt.show()

# Plot demand against time of year
plt.scatter(energy_data['TimeOfYear'], energy_data['total load actual'], s=1)
plt.xlabel('Time of Year')
plt.ylabel('Demand')
plt.title('Demand vs Time of Year')
plt.show()

# Plot average demand by month
energy_data['Month'] = energy_data.index.month
energy_data.groupby('Month')['total load actual'].mean().plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Average Demand (MW)')
plt.title('Average Demand by Month (2015-2019)')
plt.show()

# For each hour of the day, calculate and plot average demand
energy_data['Hour'] = energy_data.index.hour
energy_data.groupby('Hour')['total load actual'].mean().plot(kind='bar')
plt.xlabel('Hour')
plt.ylabel('Average Demand (MW)')
plt.title('Average Demand by Hour (2015-2019)')
plt.show()



In [ ]:
# Model

# Prepare data for modeling
X = energy_data.drop(['total load actual', 'date'], axis=1)
y = energy_data['total load actual']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Linear Regression
lm = LinearRegression()
lm.fit(X_train, y_train)
lm_error = mean_squared_error(y_test, lm.predict(X_test)) / np.mean(y_test)

# Random Forest
rf = RandomForestRegressor(n_estimators=200, random_state=1)
rf.fit(X_train, y_train)
rf_error = mean_squared_error(y_test, rf.predict(X_test)) / np.mean(y_test)

# Gradient Boosting
gb = GradientBoostingRegressor(n_estimators=200, random_state=1)
gb.fit(X_train, y_train)
gb_error = mean_squared_error(y_test, gb.predict(X_test)) / np.mean(y_test)

# Support Vector Regression
svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr.fit(X_train, y_train)
svr_error = mean_squared_error(y_test, svr.predict(X_test)) / np.mean(y_test)

# Neural Networks
mlp = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=500, alpha=0.0001, solver='adam', random_state=21)
mlp.fit(X_train, y_train)
mlp_error = mean_squared_error(y_test, mlp.predict(X_test)) / np.mean(y_test)

# XGBoost
xgb = XGBRegressor(n_estimators=200, random_state=1)
xgb.fit(X_train, y_train)
xgb_error = mean_squared_error(y_test, xgb.predict(X_test)) / np.mean(y_test)

# CatBoost
cat = CatBoostRegressor(n_estimators=200, random_state=1)
cat.fit(X_train, y_train)
cat_error = mean_squared_error(y_test, cat.predict(X_test)) / np.mean(y_test)

# PCA
pca = PCA(n_components=10)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Random Forest with PCA
rf_pca = RandomForestRegressor(n_estimators=200, random_state=1)
rf_pca.fit(X_train_pca, y_train)
rf_pca_error = mean_squared_error(y_test, rf_pca.predict(X_test_pca)) / np.mean(y_test)

# XGBoost with PCA
xgb_pca = XGBRegressor(n_estimators=200, random_state=1)
xgb_pca.fit(X_train_pca, y_train)
xgb_pca_error = mean_squared_error(y_test, xgb_pca.predict(X_test_pca)) / np.mean(y_test)

# Plotting the errors of different models
errors = [lm_error, rf_error, cat_error, gb_error, mlp_error, svr_error, xgb_error]
models = ['Linear Regression', 'Random Forest', 'CatBoost', 'Gradient Boosting', 'Neural Networks', 'Support Vector Regression', 'XGBoost']
plt.figure(figsize=(10,5))
plt.plot(models, errors)
plt.xticks(rotation=270)
plt.title('Mean Squared Error of Each Model')
plt.xlabel('Models')
plt.ylabel('Error')
plt.show()
